In [1]:
import pandas as pd
import json
import numpy as np
from collections import Counter

In [2]:
# df3 = pd.read_csv('../ess/ess3/ess3.csv')
# df4 = pd.read_csv('../ess/ess4/ess4.csv')
# df5 = pd.read_csv('../ess/ess5/ess5.csv')
# df6 = pd.read_csv('../ess/ess6/ess6.csv')
# df7 = pd.read_csv('../ess/ess7/ess7.csv')
# df8 = pd.read_csv('../ess/ess8/ess8.csv')
# df9 = pd.read_csv('../ess/ess9/ess9.csv')
# df10 = pd.read_csv('../ess/ess10/ess10.csv')
# df10S = pd.read_csv('../ess/ess10S/ess10S.csv')
# # print(df.head())
# dataframes = {'df3': df3, 'df4': df4, 'df5': df5, 'df6': df6, 'df7': df7, 'df8': df8, 'df9': df9, 'df10': df10, 'df10S': df10S}

df = pd.read_csv('../ess/all/ess_all/ess_all.csv')

In [3]:
print(df['name'].unique())

['ESS1e06_7' 'ESS2e03_6' 'ESS3e03_7' 'ESS4e04_6' 'ESS5e03_5' 'ESS6e02_6'
 'ESS7e02_3' 'ESS8e02_3' 'ESS9e03_2' 'ESS10e03_2' 'ESS10SCe03_1']


In [4]:
# Define the columns to clean
# columns_to_clean = ['lrscale', 'stfdem', 'stfeco', 'stfedu', 'stfgov', 'stfhlth', 'stflife', 'trstep', 'trstplt', 'trstprl', 'trstprt', 'atchctr', 'atcherp', 'happy']
columns_to_clean = ['lrscale', 'stfdem', 'stfeco', 'stfedu', 'stfgov', 'stfhlth', 'stflife', 'trstep', 'trstplt', 'trstprl', 'trstprt', 'happy']

# Define the values to remove
values_to_remove = [77, 88, 99, '*', np.nan]

# cleaned_dfs = []
# for name, df in dataframes.items():
#     for column in columns_to_clean:
#         df = df[~df[column].isin(values_to_remove)]
#     cleaned_dfs.append(df)
#     # print(f"{name} - {df['name']}")
# df = pd.concat(cleaned_dfs, ignore_index=True)

for column in columns_to_clean:
    df = df[~df[column].isin(values_to_remove)]

In [5]:
print(df['name'].unique())

['ESS2e03_6' 'ESS3e03_7' 'ESS4e04_6' 'ESS5e03_5' 'ESS6e02_6' 'ESS7e02_3'
 'ESS8e02_3' 'ESS9e03_2' 'ESS10e03_2' 'ESS10SCe03_1']


In [9]:
def assign_leaning(value):
    if 0 <= value <= 1:
        return 'far-left'
    elif 2 <= value <= 3:
        return 'left'
    elif value == 4:
        return 'center-left'
    elif value == 5:
        return 'center'
    elif value == 6:
        return 'center-right'
    elif 7 <= value <= 8:
        return 'right'
    elif 9 <= value <= 10:
        return 'far-right'

df['leaning'] = df['lrscale'].apply(assign_leaning)

# year
version_year_dict = {1: "2002", 2: "2004", 3: "2006", 4: "2008", 5: "2010", 6: "2012", 7: "2014", 8: "2016", 9: "2018", 10: "2020"}
df['year'] = None
for i, row in df.iterrows():
    version_str = row['name'].split('ESS')[1].split('e')[0]
    version = 10 if version_str == "10SC" else int(version_str)
    year = version_year_dict.get(version)
    df.at[i,'year'] = year

# satisfaction
satisfaction_columns = ['stfdem', 'stfeco', 'stfedu', 'stfgov', 'stfhlth', 'stflife']
df['satisfaction'] = np.round(df[satisfaction_columns].mean(axis=1),0)

# trust country
trust_columns = ['trstplt', 'trstprl', 'trstprt']
df['trust_country'] = np.round(df[trust_columns].mean(axis=1),0)

# trust eu
df['trust_eu'] = df['trstep']

# # attachment country
# df['attachment_country'] = df['atchctr']

# # attachment eu
# df['attachment_eu'] = df['atcherp']

# new_df = df[['year','cntry','leaning','happy','satisfaction','trust_country','trust_eu','attachment_country', 'attachment_eu']]
new_df = df[['year','cntry','leaning','happy','satisfaction','trust_country','trust_eu']]


In [10]:
be_years = new_df[new_df['cntry'] == 'BE']['year'].unique()
print(be_years)

['2004' '2006' '2008' '2010' '2012' '2014' '2016' '2018' '2020']


In [11]:
new_df.loc[new_df['cntry'] == 'GR', 'cntry'] = 'EL'

In [12]:
with open("../json/combined_data.json") as f:
    data = json.load(f)

In [13]:
# Initialize 'ess_data' in each country
for country in data:
    data[country]['ess_data'] = {}
print(data['BE'])

{'election_data': {'2019': {'NVA': 16.0, 'VB': 12.0, 'PS - Belgium': 9.5, 'CDV': 8.9, 'PVDA - Belgium': 8.6, 'OVLD': 8.5, 'MR': 7.6, 'SPA': 6.7, 'Ecolo': 6.1, 'Groen': 6.0, 'CDH': 3.7, 'DeFI': 2.2, 'PP - Belgium': 1.1, 'Other parties': 3.1}, '2014': {'NVA': 20.3, 'PS - Belgium': 11.7, 'CDV': 11.6, 'OVLD': 9.8, 'MR': 9.6, 'SPA': 8.8, 'Groen': 5.3, 'CDH': 5.0, 'VB': 3.7, 'PVDA - Belgium': 3.7, 'Ecolo': 3.3, 'DeFI': 1.8, 'PP - Belgium': 1.5, 'LD': 0.4, 'Other parties': 3.5}, '2010': {'NVA': 17.4, 'PS - Belgium': 13.7, 'CDV': 10.9, 'MR': 9.3, 'SPA': 9.2, 'OVLD': 8.6, 'VB': 7.8, 'CDH': 5.5, 'Ecolo': 4.8, 'Groen': 4.4, 'LD': 3.7, 'PVDA - Belgium': 1.6, 'PP - Belgium': 1.3, 'FN - Belgium': 0.5, 'Other parties': 1.3}, '2007': {'CDV/NVA': 18.5, 'MR': 12.5, 'VB': 12.0, 'OVLD': 11.8, 'PS - Belgium': 10.9, 'SPA': 10.3, 'CDH': 6.1, 'Ecolo': 5.1, 'LD': 4.0, 'Groen': 4.0, 'FN - Belgium': 2.0, 'PVDA - Belgium': 0.8, 'Other parties': 2.0}, '2003': {'OVLD': 15.3, 'SPA': 14.9, 'CDV': 13.2, 'PS - Belgium'

In [14]:
# # Iterate over the DataFrame
# for index, row in new_df.iterrows():
#     # Get country and year
#     country = row['cntry']
#     year = row['year']

#     # Initialize year in 'ess_data' if not already present
#     if year not in data[country]['ess_data']:
#         data[country]['ess_data'][year] = {
#             'leaning': Counter(),
#             'happy': Counter(),
#             'satisfaction': Counter(),
#             'trust_country': Counter(),
#             'trust_eu': Counter()
#         }

#     # Append data to 'ess_data'
#     data[country]['ess_data'][year]['leaning'].update([row['leaning']])
#     data[country]['ess_data'][year]['happy'].update([row['happy']])
#     data[country]['ess_data'][year]['satisfaction'].update([row['satisfaction']])
#     data[country]['ess_data'][year]['trust_country'].update([row['trust_country']])
#     data[country]['ess_data'][year]['trust_eu'].update([row['trust_eu']])

# Iterate over the DataFrame
for index, row in new_df.iterrows():
    # Get country and year
    country = row['cntry']
    year = row['year']

    # Initialize year in 'ess_data' if not already present
    if year not in data[country]['ess_data']:
        data[country]['ess_data'][year] = {
            'leaning': [],
            'happy': [],
            'satisfaction': [],
            'trust_country': [],
            'trust_eu': []
        }

    # Append data to 'ess_data'
    data[country]['ess_data'][year]['leaning'].append(row['leaning'])
    data[country]['ess_data'][year]['happy'].append(row['happy'])
    data[country]['ess_data'][year]['satisfaction'].append(row['satisfaction'])
    data[country]['ess_data'][year]['trust_country'].append(row['trust_country'])
    data[country]['ess_data'][year]['trust_eu'].append(row['trust_eu'])

In [15]:
from collections import OrderedDict
# Sort 'ess_data' by year
for country in data:
    data[country]['ess_data'] = OrderedDict(sorted(data[country]['ess_data'].items(), reverse=True))

In [16]:
for country in data:
    for year in data[country]['ess_data']:
        print(f"{country} {year}: {len(data[country]['ess_data'][year]['happy'])}")
    print()

AT 2020: 1836
AT 2018: 2026
AT 2016: 1699
AT 2014: 1452
AT 2006: 1715
AT 2004: 1685

BE 2020: 1210
BE 2018: 1619
BE 2016: 1664
BE 2014: 1647
BE 2012: 1752
BE 2010: 1576
BE 2008: 1599
BE 2006: 1620
BE 2004: 1461

DE 2020: 7651
DE 2018: 2123
DE 2016: 2607
DE 2014: 2691
DE 2012: 2560
DE 2010: 2471
DE 2008: 2246
DE 2006: 2266
DE 2004: 2281

DK 2018: 1329
DK 2014: 1316
DK 2012: 1417
DK 2010: 1327
DK 2008: 1342
DK 2006: 1254
DK 2004: 1169

EL 2020: 2263
EL 2010: 1796
EL 2008: 1608
EL 2004: 1595

ES 2020: 2079
ES 2018: 1254
ES 2016: 1470
ES 2014: 1408
ES 2012: 1560
ES 2010: 1525
ES 2008: 1662
ES 2006: 1349
ES 2004: 1152

FI 2020: 1463
FI 2018: 1600
FI 2016: 1803
FI 2014: 1893
FI 2012: 2001
FI 2010: 1695
FI 2008: 1964
FI 2006: 1739
FI 2004: 1783

FR 2020: 1585
FR 2018: 1677
FR 2016: 1835
FR 2014: 1697
FR 2012: 1789
FR 2010: 1572
FR 2008: 1843
FR 2006: 1758
FR 2004: 1559

IE 2020: 1293
IE 2018: 1721
IE 2016: 1976
IE 2014: 1694
IE 2012: 1984
IE 2010: 1854
IE 2008: 1521
IE 2006: 1275
IE 2004: 161

In [17]:
with open('ess_data_all.json', 'w') as f:
    json.dump(data, f, indent=3)